### 1. Import Libraries

In [4]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import *
import seaborn as sns

### 2. Readin Data

In [5]:
def combine_meta_data(tran_url,url_,url_meta_android,url_application):
    
    

IndentationError: expected an indented block (<ipython-input-5-dfbbdcb752be>, line 3)

In [6]:
# transaction table
df_tran = pd.read_table('../data/000_sample1.gz',
                       compression='gzip',
                       sep='\t',
                       quotechar='"',
                       error_bad_lines=False)

In [7]:
# creative table
df_creative = pd.read_table('../data/creatives000',
                       #compression='gzip',
                       sep='|',
                       quotechar='"',
                       error_bad_lines=False,header=None)
df_creative.columns = ['creative_id','created_at','application_id','name','video_id',
                       'is_streaming','language','enforce_language_match','device_type_whitelist',
                      'delivery_frequency_cap','time_to_show_countdown','time_to_show_close_button',
                      'cloudux_template_id']

In [8]:
# creative tag table
df_creative_tag = pd.read_table('../data/creative_tags000',
                       #compression='gzip',
                       sep='|',
                       quotechar='"',
                       error_bad_lines=False,header=None)
df_creative_tag.columns = ['creative_id','creative_tag']

In [9]:
# vidoe table
df_video = pd.read_table('../data/video_files000',
                       #compression='gzip',
                       sep='|',
                       quotechar='"',
                       error_bad_lines=False,header=None)
df_video.columns = ['video_id','size','id','width','height','duration']

### 3. Join Tables

In [10]:
df_tran['is_install'] = df_tran['timestamp_at_install'].notnull() * 1
df_tran_sub = df_tran.loc[:,['is_install','creative_id','time_of_this_impression']]

In [11]:
df_tran_ad = pd.merge(df_tran_sub, df_creative, how='left', on='creative_id', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False)
df_tran_ad_tag = pd.merge(df_tran_ad, df_creative_tag, how='left', on='creative_id', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False)
df_tran_ad_tag_video = pd.merge(df_tran_ad_tag, df_video, how='left', on='video_id', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False)

In [12]:
# check NA
# df_tran_ad_tag_video.isnull().sum()

In [13]:
df_tran_ad_tag_video.columns

Index([u'is_install', u'creative_id', u'time_of_this_impression',
       u'created_at', u'application_id', u'name', u'video_id', u'is_streaming',
       u'language', u'enforce_language_match', u'device_type_whitelist',
       u'delivery_frequency_cap', u'time_to_show_countdown',
       u'time_to_show_close_button', u'cloudux_template_id', u'creative_tag',
       u'size', u'id', u'width', u'height', u'duration'],
      dtype='object')

### 4. Add Y & Clean Data

In [ ]:
df_tran_ad_tag_video['language'] = df_tran_ad_tag_video['language'].apply(lambda x: str(x).lower().split('-')[0])
df_tran_ad_tag_video['is_install'] = df_tran_ad_tag_video['is_install'].astype('category')

### 5. Create new features

#### Ad Age

In [ ]:
df_tran_ad_tag_video['time_delta'] = pd.to_datetime(df_tran_ad_tag_video['time_of_this_impression'])-pd.to_datetime(df_tran_ad_tag_video['created_at'])

In [ ]:
def to_day(x):
    if isinstance(x,pd.tslib.Timedelta):
        return x.days
    else:
        return np.nan

In [ ]:
df_tran_ad_tag_video['time_delta_days'] = df_tran_ad_tag_video['time_delta'].apply(to_day)

In [ ]:
df_tran_ad_tag_video['time_delta_days'] = df_tran_ad_tag_video['time_delta'].dt.day

### 5. Continous Variables

In [ ]:
# continous_EDA function
def continous_EDA(col,df):
    group0 = df[df['is_install']==0][col].dropna()
    group1 = df[df['is_install']==1][col].dropna()
    
    # T-test
    print 't-test'
    p = ttest_ind(group0, group1, axis=0, equal_var=False)

    # histogram of two groups
    print 'Distribution of two groups'
    fig, ax = plt.subplots()
    i = 0
    for a in [group0, group1]:
        sns.distplot(a, ax=ax, kde=False,label=str(i))
        i += 1
    ax.legend(loc=2,prop={'size':7})
    plt.show()
    
    # Box-plot
    print 'Box-plot'
    ax = sns.boxplot(x="is_install", y=col, data=df)
    plt.show()
    return p

In [ ]:
cont_cols = ['time_delta_days','size','width','height','duration']
for col in cont_cols:
    print '-----------Feature:', col,'----------------'
    continous_EDA(col,df_tran_ad_tag_vide)
    p_lst.append(p)
    print '---------------------------------\n'

In [ ]:
significant_cols = [col for col,p in zip(cont_cols,p_lst) if p<0.05]
'significant columns:', significant_cols

### 6. Categorical Variables

In [ ]:
cat_cols = ['is_streaming','language','enforce_language_match','device_type_whitelist',
            'delivery_frequency_cap','time_to_show_countdown','time_to_show_close_button',
            'cloudux_template_id','creative_tag']

In [ ]:
def categotical_EDA(col, df):
    count = pd.crosstab(df['is_install'],df[col])
    count = count.iloc[:,0:]
    pcts = count.div(count.sum(1).astype(float),axis=0)
    g, p, dof, expctd = chi2_contingency(pcts.as_matrix())
    
    ax = pcts.plot(kind='barh',stacked=True, alpha=0.5, figsize=(8,6))
    ax.set_xlabel("Percentage", fontsize=12)
    ax.set_ylabel("Is Install", fontsize=12)
    ax.legend(loc=2,prop={'size':7})
    
    plt.show()
    return p

In [ ]:
p_lst = []
for col in cat_cols:
    print '-----------Feature:', col,'----------------'
    p = categotical_EDA(col,df_tran_ad_tag_vide)
    p_lst.append(p)
    print '---------------------------------\n'

In [ ]:
significant_cols = [col for col,p in zip(cat_cols,p_lst) if p<0.05]
'significant columns:', significant_cols

### 7. Conclusion
Useful Features:
* Continous Variable: ['time_delta_days']
